In [35]:
pip install hdfs

Note: you may need to restart the kernel to use updated packages.


### Script to export the data from local to HDFS

In [37]:
import os
from hdfs import InsecureClient

# HDFS client
hdfs_client = InsecureClient('http://100.126.132.104:9870', user='hadoop')

# Path to the folder containing files
local_data_folder = 'C:/Users/Manoj/Downloads/DE_SCRAPPING'
# Path in HDFS to store the files
hdfs_target_folder = '/home/hadoop/data/nameNode'

# Ensuring the HDFS target folder exists
hdfs_client.makedirs(hdfs_target_folder)

# Uploading the entire folder to HDFS 
try:
    hdfs_client.upload(hdfs_target_folder, local_data_folder, overwrite=True)
    print(f'Successfully uploaded folder {local_data_folder} to {hdfs_target_folder}')
except Exception as e:
    print(f'Failed to upload folder {local_data_folder}: {e}')


Successfully uploaded folder C:/Users/Manoj/Downloads/DE_SCRAPPING to /home/hadoop/data/nameNode


### Script to ingest the data to SQL

In [44]:
import pandas as pd
from hdfs import InsecureClient
from sqlalchemy import create_engine
import os

# HDFS connection detailsv (master-node,user)
hdfs_host = 'http://100.126.132.104:9870'
hdfs_user = 'hadoop'
hdfs_client = InsecureClient(hdfs_host, user=hdfs_user)

# List of tables and their corresponding HDFS paths
hdfs_directory = "/home/hadoop/data/nameNode/DE_SCRAPPING/"
tables = {
    'TeamData': hdfs_directory + 'teamData.csv',
    'PlayerInfo': hdfs_directory + 'playerinformation.csv',
    'GroundInfo': hdfs_directory + 'groundDetails.csv',
    'MatchSummary': hdfs_directory + 'matchsummary.csv',
    'MatchDetails': hdfs_directory + 'matchDetails.csv',
    'BattingSummary': hdfs_directory + 'battingSummary.csv',
    'BowlingSummary': hdfs_directory + 'bowlingsummary.csv',
}

# MySQL database connection details
db_username = 'root'
db_password = '123456'
db_host = '192.168.56.101'
db_port = '3306'  # Default MySQL port
db_name = 't20'

# Create a connection string based on the details above
connection_string = f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Function to read CSV from HDFS and convert to Pandas DataFrame
def read_hdfs_csv(hdfs_path):
    with hdfs_client.read(hdfs_path, encoding='utf-8') as reader:
        return pd.read_csv(reader)

# Iterate over the tables dictionary
for table_name, hdfs_path in tables.items():
    try:
        if hdfs_path.endswith('.csv'):
            # Read CSV file from HDFS into a Pandas DataFrame
            pandas_df = read_hdfs_csv(hdfs_path)
        else:
            print(f"Unsupported file type: {hdfs_path}")
            continue

        # Ingest data into SQL table
        pandas_df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        print(f'Successfully inserted data into {table_name}')
    except Exception as e:
        print(f'Failed to insert data into {table_name}: {e}')


Successfully inserted data into TeamData
Successfully inserted data into PlayerInfo
Successfully inserted data into GroundInfo
Successfully inserted data into MatchSummary
Successfully inserted data into MatchDetails
Successfully inserted data into BattingSummary
Successfully inserted data into BowlingSummary
